In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("EXJPUS.csv")
tsValues = data['EXJPUS']
import math
tsValuesLog = [math.log(ele) for ele in tsValues]

In [2]:
PredictionOutputMAE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
PredictionOutputRMSE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
startPrediction = 100
totalData = len(tsValuesLog)

In [3]:
def mean_error(pred,real):
    return np.mean([pred[i]-real[i] for i in range(len(pred))])
def mean_absolute_error(pred,real):
    return np.mean([np.abs(pred[i]-real[i]) for i in range(len(pred))])
def root_mean_square_error(pred,real):
    return np.sqrt(np.mean([pow(pred[i]-real[i],2) for i in range(len(pred))]))

In [4]:
totalData

173

In [5]:
startPrediction

100

In [6]:
#benchmark
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        pred.append(tsValuesLog[startPrediction-1+i])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
PredictionOutputMAE['RandomWalk'] = mae_
PredictionOutputRMSE['RandomWalk'] = rmse_

In [9]:
from statsmodels.tsa.arima_model import ARMA
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
        except ValueError:
            pred.append(tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [10]:
PredictionOutputRMSE

,RandomWalk,AR_Diff_1_1
monthly,0.021630,0.021497
3_month,0.044147,0.045363
6_month,0.065965,0.069422
12_month,0.093917,0.100575


In [14]:
PredictionOutputMAE

,RandomWalk
monthly,0.016524
3_month,0.033247
6_month,0.050263
12_month,0.071854


In [15]:
#test performance random walk with drift

mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        monthly_change = [tsValuesLog[j]-tsValuesLog[j-l] for j in range(i+l,i+startPrediction)]
        mean_monthly_change = np.mean(monthly_change)
        pred.append(tsValuesLog[startPrediction+i-1]+mean_monthly_change)
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['RandomWalkWithDrift'] = mae_
PredictionOutputRMSE['RandomWalkWithDrift'] = rmse_

In [12]:
PredictionOutputRMSE

,RandomWalk,AR_Diff_1_1,AR_Diff_Mean
monthly,0.021630,0.021497,0.021444
3_month,0.044147,0.045363,0.044647
6_month,0.065965,0.069422,0.068485
12_month,0.093917,0.100575,0.099899


In [11]:
from statsmodels.tsa.arima_model import ARMA
mae_ = []
rmse_ = []
test_queue = [[1,1],[0,1],[1,0]]
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        single_pred = []
        for [p,q] in test_queue:
            try:
                model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
                result = model.fit()
                single_pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
            except ValueError:
                single_pred.append(tsValuesLog[i+startPrediction-1])
        pred.append(np.mean(single_pred))
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_Mean'] = mae_
PredictionOutputRMSE['AR_Diff_Mean'] = rmse_

C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead o

In [17]:
from statsmodels.tsa.arima_model import ARMA
#out-of-sample-test
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

In [20]:
mae_ = []
rmse_ = []
p=2
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
        except ValueError:
            pred.append(tsValuesLog[startPrediction+i-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1127: RuntimeWarning: invalid value encountered in sqrt
  return rho, np.sqrt(sigmasq)


In [18]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1
monthly,0.016524,0.016680,0.016592
3_month,0.033247,0.034238,0.032933
6_month,0.050263,0.054264,0.049131
12_month,0.071854,0.083193,0.071525


In [8]:
tsValuesLog_d_1 = [0]
tsValuesLog_d_1 += [tsValuesLog[i+1]-tsValuesLog[i] for i in range(totalData-1)]

In [22]:
mae_ = []
rmse_ = []
p=1
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [23]:
mae_ = []
rmse_ = []
p=0
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [32]:
#Unobserved model
import statsmodels.api as sm

class UnobservedComponentModel:
    def __init__(self):
        self.model_name = "UnobservedComponentModel"
        return

    def fit(self, ts):
        unrestricted_model = {
            'level': 'local linear trend', 'cycle': False , 'seasonal': 12
        }

        model = sm.tsa.UnobservedComponents(endog=ts, **unrestricted_model)
        self.trained_model = model.fit()
        return self

    def predict(self, next_n_prediction):
        prediction = self.trained_model.forecast(steps=next_n_prediction)
        return prediction

In [33]:
%%time
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-(l-1)):
        model = UnobservedComponentModel()
        model.fit(tsValuesLog[i:i+startPrediction])
        pred.append(model.predict(l)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check

Wall time: 2min 50s


In [30]:
PredictionOutputMAE["UnobservedComponentModel"] = mae_
PredictionOutputRMSE["UnobservedComponentModel"] = rmse_

In [37]:
PredictionOutputMAE["UnobservedComponentModel_With_Season"] = mae_
PredictionOutputRMSE["UnobservedComponentModel_With_Season"] = rmse_

In [38]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.016524,0.016680,0.016592,0.017002,0.016997,0.016736,0.018231,0.021146
3_month,0.033247,0.034238,0.032933,0.033280,0.033803,0.033706,0.042167,0.050248
6_month,0.050263,0.054264,0.049131,0.049585,0.052650,0.052546,0.075615,0.088664
12_month,0.071854,0.083193,0.071525,0.073674,0.077504,0.077466,0.128358,0.128643


In [39]:
PredictionOutputRMSE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.021630,0.021918,0.021245,0.021344,0.021468,0.021538,0.023149,0.025753
3_month,0.044147,0.045611,0.043230,0.043472,0.044622,0.044499,0.055462,0.063593
6_month,0.065965,0.069779,0.064976,0.065447,0.068508,0.068110,0.096912,0.109107
12_month,0.093917,0.104408,0.091449,0.091892,0.099868,0.099785,0.166532,0.164652


In [40]:
PredictionOutputMAE.to_csv("EXJPUS_MAE.csv")
PredictionOutputRMSE.to_csv("EXJPUS_RMAE.csv")